# Modelling opinion shifts with the Smooth Bounded Confidence Model

Salcedo, Dylan Nicolas Emmanuel \
2011-24175 \
Physics 215 \
Mini-Project

In this project, we investigate the effect of changing parameters on the Smooth Bounded Confidence Model as applied to opinions of individuals in social networks.

Main reference: https://arxiv.org/abs/cond-mat/0410199

## Setup

We will be using Julia's mutables structs to create our models and agents (nodes). The model consists of all the agents present as an array, with each agent containing the attributes `id`, `opinion`, and `uncertainty`.

In [1]:
mutable struct Agent
    id::Int64
    opinion::Float64
    uncertainty::Float64
end

mutable struct Model
    agents::Array
end

Unlike other Object-Oriented Programming languages, Julia doesn't exactly have dedicated methods for its structs. Instead, we will be using functions to operate on them.

Our models will consist of the following agents:

- Lower extremists: 5% of all agents, with initial opinion -1.0 and uncertainty 0.05
- Upper extremists: 5% of all agents, with initial opinion +1.0 and uncertainty 0.05
- Moderates: The rest of the agents, with initial opinion randomly chosen from -1.0 to +1.0. Uncertainties vary per model

In [2]:
function randRange(a,b)
    return rand()*(b-a) + a
end

function createModel(model,num_agents,mod_uncertainty)
    all_agents = []
    num_extremists = Int(floor(0.05*num_agents))
    num_moderates = num_agents - 2*num_extremists
    id_num = 1

    # Create the lower extremists
    for agent in 1:num_extremists
        all_agents = vcat(
            all_agents,
            Agent(
                id_num,
                -1.0,
                0.05
            )
        )
        id_num = id_num + 1
    end

    # Create the upper extremists
    for agent in 1:num_extremists
        all_agents = vcat(
            all_agents,
            Agent(
                id_num,
                1.0,
                0.05
            )
        )
        id_num = id_num + 1
    end

    # Create moderates
    for agent in 1:num_moderates
        all_agents = vcat(
            all_agents,
            Agent(
                id_num,
                randRange(-1.0,1.0),
                mod_uncertainty
            )
        )
        id_num = id_num + 1
    end

    model.agents = all_agents
    return model
end

createModel (generic function with 1 method)

This project assumes that all nodes are connected to each other (i.e. fully connected network). For each node-neighbor interaction, the node's opinion and uncertainty changes according to the following,

New opinion: $\frac{x + y*g(x-y)}{1 + g(x-y)}$

New uncertainty: $\frac{u + v*g(x-y)}{1 + g(x-y)}$

$g(x-y) = exp(-(\frac{x-y}{u})^2)$

where
- x: node opinion
- u: node uncertainty
- y: neighbor opinion
- v: neighbor uncertainty

In [3]:
function kernel(x,y,u)
    g = exp(-((x-y)/u)^2)
    return g
end

function interact(agent1,agent2)
    x = agent1.opinion
    y = agent2.opinion
    u = agent1.uncertainty
    v = agent2.uncertainty

    x2 = (x + y*kernel(x,y,u)) / (1 + kernel(x,y,u))
    u2 = (u + v*kernel(x,y,u)) / (1 + kernel(x,y,u))
    return [x2, u2]
end

function evolveAgents(model)
    all_agents = []

    for agent in model.agents
        y = rand(1:length(model.agents))
        while y == agent.id
            y = rand(1:length(model.agents))
        end
        neighbor = model.agents[y]
        
        new_agent = interact(agent,neighbor)

        all_agents = vcat(
            all_agents,
            Agent(
                agent.id,
                new_agent[1],
                new_agent[2]
            )
        )
    end

    model.agents = all_agents
    return model
end

evolveAgents (generic function with 1 method)

For this project, we will be creating three different models with varying initial uncertainties for their moderates (0.1,0.5,1.2). All models have 100 agents and are allowed to evolve for 500 timesteps.

We will be using Julia's `Plots` package to create our scatter plots

In [4]:
using Plots;

## u(0) = 0.1: Central convergence

In [5]:
MainModel_1 = Model([]);
MainModel_1 = createModel(MainModel_1,100,0.1);

xs_1 = []
ys_1 = []
us_1 = []

for generation in 1:500
    if generation != 1
        MainModel_1 = evolveAgents(MainModel_1)
    end
    for i in 1:length(MainModel_1.agents)
        xs_1 = vcat(xs_1,generation)
        ys_1 = vcat(ys_1,MainModel_1.agents[i].opinion)
        us_1 = vcat(us_1,MainModel_1.agents[i].uncertainty)
    end
end

In [9]:
plot1 = scatter(
    xs_1,
    ys_1,
    markersize=2.0,
    markerstrokewidth=0.2,
    legend=false,
    xlims=(0,500),
    size=(1600,900),
    title="Opinion dynamics: u(0) = 0.1",
    xlabel="Timestep",
    ylabel="Opinions"
);
savefig(plot1,"u_01.png")

<img src="./u_01.png" style="height: 450px; width:800px;">

Due to the small uncertainties, once nodes are clustered, they wouldnt be able to interact with other clusters, resulting in a more spread-out distribution.

## u(0) = 0.5: Double convergence

In [12]:
MainModel_2 = Model([]);
MainModel_2 = createModel(MainModel_2,100,0.5);

xs_2 = []
ys_2 = []
us_2 = []

for generation in 1:500
    if generation != 1
        MainModel_2 = evolveAgents(MainModel_2)
    end
    for i in 1:length(MainModel_2.agents)
        xs_2 = vcat(xs_2,generation)
        ys_2 = vcat(ys_2,MainModel_2.agents[i].opinion)
        us_2 = vcat(us_2,MainModel_2.agents[i].uncertainty)
    end
end

In [13]:
plot2 = scatter(
    xs_2,
    ys_2,
    markersize=2.0,
    markerstrokewidth=0.2,
    legend=false,
    xlims=(0,500),
    size=(1600,900),
    title="Opinion dynamics: u(0) = 0.5",
    xlabel="Timestep",
    ylabel="Opinions"
)
savefig(plot2,"u_05.png")

<img src="./u_05.png" style="height: 450px; width:800px;">

In this model, node uncertainties are large enough to form clusters with one on each extreme. Once these two clusters form, they are too far apart to interact with each other.

## u(0) = 1.2: Single convergence

In [14]:
MainModel_3 = Model([]);
MainModel_3 = createModel(MainModel_3,100,1.2);

xs_3 = []
ys_3 = []
us_3 = []

for generation in 1:500
    if generation != 1
        MainModel_3 = evolveAgents(MainModel_3)
    end
    for i in 1:length(MainModel_3.agents)
        xs_3 = vcat(xs_3,generation)
        ys_3 = vcat(ys_3,MainModel_3.agents[i].opinion)
        us_3 = vcat(us_3,MainModel_3.agents[i].uncertainty)
    end
end

In [15]:
plot3 = scatter(
    xs_3,
    ys_3,
    markersize=2.0,
    markerstrokewidth=0.2,
    legend=false,
    xlims=(0,500),
    size=(1600,900),
    title="Opinion dynamics: u(0) = 1.2",
    xlabel="Timestep",
    ylabel="Opinions"
)
savefig(plot3,"u_12.png")

<img src="./u_12.png" style="height: 450px; width:800px;">

With the uncertainties large enough, even after two extreme clusters are formed, majority of the nodes can still interact with both sides, resulting in them getting gravitated towards the more "popular" extreme.